In [1]:
import torch
print("Using torch", torch.__version__)

Using torch 2.0.1+cu117


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


## Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split    # For splitting the dataset
from sklearn import svm
from sklearn.datasets import make_moons
from sklearn.svm import SVC
from sklearn.inspection import DecisionBoundaryDisplay  # For plotting the non liner decision boundaries
from sklearn.model_selection import GridSearchCV        # For hyperperameter tuning

from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## Data Pre-processing

In [4]:
training_data_95 = pd.read_csv('./csvs/hog_95/extracted_features_hog_95.csv',header = None)
print(training_data_95)

# X_train_df = training_data_95.drop(columns = [0])
# y_train_df = training_data_95[0]

# print(X_train_df)
# print(y_train_df)

                    0         1         2         3         4         5     \
0      Rachel_Leigh_Cook  0.101289  1.330157 -0.964354  0.278769 -0.543510   
1          Roseanne_Barr  0.391125  0.704784  1.217563 -0.780129 -1.414380   
2         Vladimir_Putin -1.246895 -0.680080  0.591671 -2.354134 -1.920589   
3           Carlos_Menem  0.597257 -0.661087  1.356273 -0.878202 -0.588220   
4          Lynne_Thigpen -1.589231 -0.049223 -1.306154 -0.461846  0.250779   
...                  ...       ...       ...       ...       ...       ...   
10581    Rohinton_Mistry  1.724274 -1.189989 -1.052075  1.067114 -0.041331   
10582      George_W_Bush  2.658667  0.404976  0.808148 -0.520925 -0.804796   
10583        Vicente_Fox  0.479659 -0.820659  0.506519  0.026553  1.815414   
10584          Todd_Wike -1.765033 -0.231035  0.285658 -1.902368 -0.086056   
10585         Kofi_Annan  1.083461  0.993570 -0.346959  1.194149 -1.330585   

           6         7         8         9     ...  3239   3240

In [5]:
testing_data_95 = pd.read_csv('./csvs/hog_95/extracted_features_test_hog_95.csv', header = None)
print(testing_data_95)

# X_test_df = testing_data_95.drop(columns = [0])
# y_test_df = testing_data_95[0]

# print(X_test_df)
# print(y_test_df)

                         0         1         2         3         4     \
0              Carolina_Kluft  1.943908 -0.685770  0.135847 -0.574862   
1            Roger_Etchegaray -1.104129  0.259980 -1.581375  0.471127   
2                  JK_Rowling  1.170456 -1.014306 -0.396331  0.125549   
3     Valery_Giscard_dEstaing  1.449810  0.777844  1.786893 -0.590618   
4         Juan_Carlos_Ferrero -0.364156 -1.960419 -0.170895  0.499791   
...                       ...       ...       ...       ...       ...   
2642            Michael_Chang  1.412572 -0.760504  0.483098 -0.281977   
2643              Osrat_Iosef  1.675409  2.553274 -0.504704 -0.422554   
2644              Yann_Martel  1.023609 -0.542472  0.853466  0.449747   
2645             Mitzi_Gaynor -0.901391  0.372586  2.181821  0.809191   
2646                Fujio_Cho  0.938650 -0.126036 -1.422143  0.285539   

          5         6         7         8         9     ...  3239   3240  \
0    -0.959423  0.472946  0.260016  0.167261 -0

### Taking only the images of persons whose number of images are greater than equal to 50

In [6]:
df = pd.concat([training_data_95,testing_data_95],axis = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13233 entries, 0 to 2646
Columns: 3249 entries, 0 to 3248
dtypes: float64(3248), object(1)
memory usage: 328.1+ MB


In [7]:
df_50 = df[df[0].map(df[0].value_counts()) >= 50]  # Reference Stack overflow
df_50.reset_index(drop=True, inplace = True)
df_50

,0,1,2,3,4,5,6,7,8,9,...,3239,3240,3241,3242,3243,3244,3245,3246,3247,3248
0,George_W_Bush,-1.824086,-0.401801,-0.521794,-0.587088,-0.509684,0.280968,-1.599484,0.290741,0.713462,...,31.0,359.0,1280.0,349.0,26.0,262.0,456.0,332.0,313.0,16523.0
1,Colin_Powell,0.549831,-0.715630,1.769869,0.222841,0.108078,0.219034,-0.818753,-0.839518,-0.129442,...,44.0,504.0,1817.0,748.0,30.0,501.0,556.0,396.0,362.0,14746.0
2,George_W_Bush,-0.628315,-0.750331,0.547397,0.478640,2.337426,0.526050,0.712348,-1.843158,-0.076518,...,27.0,302.0,1903.0,347.0,13.0,263.0,381.0,313.0,252.0,29224.0
3,George_W_Bush,2.176470,-0.997268,0.699885,-1.213921,0.090995,1.768202,0.787791,-0.212840,0.012334,...,58.0,725.0,2027.0,597.0,39.0,361.0,507.0,334.0,312.0,6742.0
4,Tony_Blair,-2.303506,-0.646863,0.022650,1.529321,0.472518,0.093752,0.199501,-0.097458,-0.722055,...,25.0,374.0,2097.0,485.0,12.0,340.0,622.0,450.0,491.0,16778.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,George_W_Bush,-0.754818,-0.751547,-0.307345,0.836765,-0.443559,0.034152,1.786415,0.199164,-0.641788,...,41.0,496.0,1763.0,513.0,22.0,445.0,518.0,527.0,430.0,11888.0
1556,Tony_Blair,0.356975,-0.575622,-1.112081,-0.147580,2.257156,1.439668,-0.556521,0.081855,1.255535,...,45.0,646.0,3227.0,691.0,25.0,573.0,724.0,379.0,567.0,12012.0
1557,George_W_Bush,2.378569,0.299064,-0.343518,1.097499,0.782947,0.991848,-0.635729,-0.408563,-0.150484,...,40.0,353.0,1540.0,480.0,35.0,355.0,396.0,395.0,318.0,27232.0
1558,Tony_Blair,0.732607,-0.158723,-2.044487,0.405323,1.605780,-0.532496,-0.508825,0.240685,0.907796,...,35.0,522.0,1654.0,719.0,30.0,543.0,728.0,603.0,519.0,8109.0


In [8]:
print("Number of persons are ",df_50[0].unique().shape[0])

Number of persons are  12


In [9]:
X_50 = df_50.drop(columns = [0])
y_50 = df_50[0]
X_train_50, X_test_50, y_train_50, y_test_50 = train_test_split(X_50,y_50,train_size=0.8,random_state=42)
X_train_50.reset_index(drop=True, inplace = True)
X_test_50.reset_index(drop=True, inplace = True)
y_train_50.reset_index(drop=True, inplace = True)
y_test_50.reset_index(drop=True, inplace = True)

## LinearSVC

In [23]:
# Training Linear Support Vector Classifier (LinearSVC)
lin_clf = svm.LinearSVC(dual=False)
lin_clf.fit(X_train_50, y_train_50)

LinearSVC(dual=False)

In [25]:
pred_y_test_50 = lin_clf.predict(X_test_50)

In [26]:
# Accuracy of linearSVC model
linSVC_accuracy = lin_clf.score(X_test_50, y_test_50)
print("Accuracy of linearSVC is ",linSVC_accuracy)

Accuracy of linearSVC is  0.11295806573479411


In [27]:
# Classification Report of linearSVC model
print(classification_report(y_test_50,pred_y_test_50))

                   precision    recall  f1-score   support

     Ariel_Sharon       0.39      0.64      0.49        14
     Colin_Powell       0.56      0.57      0.57        47
  Donald_Rumsfeld       0.53      0.29      0.38        31
    George_W_Bush       0.67      0.76      0.71        92
Gerhard_Schroeder       0.17      0.19      0.18        21
      Hugo_Chavez       0.14      0.10      0.11        21
   Jacques_Chirac       0.14      0.15      0.15        13
    Jean_Chretien       0.20      0.18      0.19        11
    John_Ashcroft       0.00      0.00      0.00        12
Junichiro_Koizumi       0.15      0.12      0.13        17
  Serena_Williams       0.43      0.38      0.40         8
       Tony_Blair       0.23      0.24      0.24        25

         accuracy                           0.44       312
        macro avg       0.30      0.30      0.30       312
     weighted avg       0.43      0.44      0.42       312



## SVM Models with Linear, polynomial and rbf kernals

In [10]:
svm_linear = SVC(kernel='linear') # linear kernal
svm_poly = SVC(kernel='poly', degree= 3)  # Polynomial kernel of degree 3
svm_rbf = SVC(kernel='rbf')  # RBF kernel

In [11]:
svm_linear.fit(X_train_50, y_train_50)
svm_poly.fit(X_train_50, y_train_50)
svm_rbf.fit(X_train_50, y_train_50)

SVC()

### Linear Kernel

In [41]:
svm_linear.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [27]:
lin_pred_y_50 = svm_linear.predict(X_test_50)

In [28]:
lin_accuracy = accuracy_score(y_test_50,lin_pred_y_50)
print("Accuracy of SVM with Linear kernel is ",lin_accuracy)

Accuracy of SVM with RBF kernel is  0.3557692307692308


In [29]:
print(classification_report(y_test_50,lin_pred_y_50))

                   precision    recall  f1-score   support

     Ariel_Sharon       0.53      0.71      0.61        14
     Colin_Powell       0.41      0.47      0.44        47
  Donald_Rumsfeld       0.52      0.35      0.42        31
    George_W_Bush       0.43      0.57      0.49        92
Gerhard_Schroeder       0.26      0.29      0.27        21
      Hugo_Chavez       0.11      0.05      0.07        21
   Jacques_Chirac       0.00      0.00      0.00        13
    Jean_Chretien       0.17      0.09      0.12        11
    John_Ashcroft       0.14      0.08      0.11        12
Junichiro_Koizumi       0.17      0.06      0.09        17
  Serena_Williams       0.38      0.38      0.38         8
       Tony_Blair       0.11      0.12      0.11        25

         accuracy                           0.36       312
        macro avg       0.27      0.26      0.26       312
     weighted avg       0.33      0.36      0.33       312



#### Tunning Hyperparameter

In [32]:
c_values = [0.1, 1, 10, 100]

accuracy_values = []
predicted_values = []

for i in c_values:
    svm_lin_tunned = SVC(kernel='linear', C = i )
    svm_lin_tunned.fit(X_train_50, y_train_50)
    lin_t_pred_y_50 = svm_lin_tunned.predict(X_test_50)
    lin_t_accuracy = accuracy_score(y_test_50,lin_t_pred_y_50)
    
    accuracy_values.append(lin_t_accuracy)
    predicted_values.append(lin_t_pred_y_50)
    
print(accuracy_values)

[0.3557692307692308, 0.3557692307692308, 0.3557692307692308, 0.3557692307692308]


Therefor, value of c (between 0.1 to 100) is not affecting the accuracy.
So, no requirement of hyperparameter tuning.

### Polynomial Kernel

In [42]:
svm_poly.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'poly',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [15]:
poly_pred_y_50 = svm_poly.predict(X_test_50)

In [19]:
poly_accuracy = accuracy_score(y_test_50,poly_pred_y_50)
print("Accuracy of SVM with polynomial kernel (of degree 3) is ",poly_accuracy)

0.3076923076923077
Accuracy of SVM with polynomial kernel (of degree 3) is  0.3076923076923077


In [17]:
print(classification_report(y_test_50,poly_pred_y_50))

                   precision    recall  f1-score   support

     Ariel_Sharon       0.00      0.00      0.00        14
     Colin_Powell       0.43      0.13      0.20        47
  Donald_Rumsfeld       0.00      0.00      0.00        31
    George_W_Bush       0.30      0.98      0.46        92
Gerhard_Schroeder       0.00      0.00      0.00        21
      Hugo_Chavez       0.00      0.00      0.00        21
   Jacques_Chirac       0.00      0.00      0.00        13
    Jean_Chretien       0.00      0.00      0.00        11
    John_Ashcroft       0.00      0.00      0.00        12
Junichiro_Koizumi       0.00      0.00      0.00        17
  Serena_Williams       0.00      0.00      0.00         8
       Tony_Blair       0.00      0.00      0.00        25

         accuracy                           0.31       312
        macro avg       0.06      0.09      0.05       312
     weighted avg       0.15      0.31      0.17       312



C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Tuning Hyperparameters

In [43]:
# Defining parameter grid
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1]}

# Performing grid search
grid_search = GridSearchCV(estimator=svm_poly, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_50,y_train_50)

# Best parameters
poly_best_params_initial = grid_search.best_params_
poly_best_params_initial

{'C': 0.1, 'gamma': 0.001}

In [44]:
svm_poly_tunned = SVC(kernel='poly', gamma=poly_best_params_initial['gamma'], C =poly_best_params_initial['C'] )
svm_poly_tunned.fit(X_train_50, y_train_50)

SVC(C=0.1, gamma=0.001, kernel='poly')

In [45]:
poly_t_pred_y_50 = svm_poly_tunned.predict(X_test_50)

In [46]:
poly_t_accuracy = accuracy_score(y_test_50,poly_t_pred_y_50)
print("Accuracy of SVM with Polynomial kernel (degree = 3) with tunned hyperparameters is ",poly_t_accuracy)

Accuracy of SVM with Polynomial kernel (degree = 3) with tunned hyperparameters is  0.3141025641025641


In [47]:
print(classification_report(y_test_50,poly_t_pred_y_50))

                   precision    recall  f1-score   support

     Ariel_Sharon       0.24      0.50      0.33        14
     Colin_Powell       0.32      0.36      0.34        47
  Donald_Rumsfeld       0.33      0.23      0.27        31
    George_W_Bush       0.46      0.57      0.50        92
Gerhard_Schroeder       0.08      0.05      0.06        21
      Hugo_Chavez       0.10      0.05      0.06        21
   Jacques_Chirac       0.27      0.23      0.25        13
    Jean_Chretien       0.11      0.09      0.10        11
    John_Ashcroft       0.08      0.08      0.08        12
Junichiro_Koizumi       0.27      0.18      0.21        17
  Serena_Williams       0.12      0.12      0.12         8
       Tony_Blair       0.19      0.16      0.17        25

         accuracy                           0.31       312
        macro avg       0.22      0.22      0.21       312
     weighted avg       0.29      0.31      0.30       312



### RBF Kernel

In [22]:
rbf_pred_y_50 = svm_rbf.predict(X_test_50)

In [23]:
rbf_accuracy = accuracy_score(y_test_50,rbf_pred_y_50)
print("Accuracy of SVM with RBF kernel is ",rbf_accuracy)

Accuracy of SVM with RBF kernel is  0.2980769230769231


In [24]:
print(classification_report(y_test_50,rbf_pred_y_50))

                   precision    recall  f1-score   support

     Ariel_Sharon       0.00      0.00      0.00        14
     Colin_Powell       0.28      0.19      0.23        47
  Donald_Rumsfeld       0.00      0.00      0.00        31
    George_W_Bush       0.30      0.91      0.45        92
Gerhard_Schroeder       0.00      0.00      0.00        21
      Hugo_Chavez       0.00      0.00      0.00        21
   Jacques_Chirac       0.00      0.00      0.00        13
    Jean_Chretien       0.00      0.00      0.00        11
    John_Ashcroft       0.00      0.00      0.00        12
Junichiro_Koizumi       0.00      0.00      0.00        17
  Serena_Williams       0.00      0.00      0.00         8
       Tony_Blair       0.00      0.00      0.00        25

         accuracy                           0.30       312
        macro avg       0.05      0.09      0.06       312
     weighted avg       0.13      0.30      0.17       312



C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Tuning Hyperparameteres

In [48]:
# Define parameter grid
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1]}

# Performing grid search
grid_search = GridSearchCV(estimator=svm_rbf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_50, y_train_50)

# Best parameters
rbf_best_params_initial = grid_search.best_params_
rbf_best_params_initial

{'C': 0.1, 'gamma': 0.001}

In [50]:
svm_rbf_tunned = SVC(kernel='rbf', gamma=rbf_best_params_initial['gamma'], C =rbf_best_params_initial['C'] )
svm_rbf_tunned.fit(X_train_50, y_train_50)

SVC(C=0.1, gamma=0.001)

In [51]:
rbf_t_pred_y_50 = svm_rbf_tunned.predict(X_test_50)

In [52]:
rbf_t_accuracy = accuracy_score(y_test_50,rbf_t_pred_y_50)
print("Accuracy of SVM with rbf kernel with tunned hyperparameters is ",rbf_t_accuracy)

Accuracy of SVM with rbf kernel with tunned hyperparameters is  0.2948717948717949


In [53]:
print(classification_report(y_test_50,rbf_t_pred_y_50))

                   precision    recall  f1-score   support

     Ariel_Sharon       0.00      0.00      0.00        14
     Colin_Powell       0.00      0.00      0.00        47
  Donald_Rumsfeld       0.00      0.00      0.00        31
    George_W_Bush       0.29      1.00      0.46        92
Gerhard_Schroeder       0.00      0.00      0.00        21
      Hugo_Chavez       0.00      0.00      0.00        21
   Jacques_Chirac       0.00      0.00      0.00        13
    Jean_Chretien       0.00      0.00      0.00        11
    John_Ashcroft       0.00      0.00      0.00        12
Junichiro_Koizumi       0.00      0.00      0.00        17
  Serena_Williams       0.00      0.00      0.00         8
       Tony_Blair       0.00      0.00      0.00        25

         accuracy                           0.29       312
        macro avg       0.02      0.08      0.04       312
     weighted avg       0.09      0.29      0.13       312



C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [54]:
# Defining the starting and ending points for C and gamma, and the number of points
start_C = 0.1
end_C = 10
num_C = 10

start_gamma = 0.001
end_gamma = 0.1
num_gamma = 10

# Parameter grid with specified ranges and number of points
param_grid = {'C': np.linspace(start_C, end_C, num_C),
              'gamma': np.linspace(start_gamma, end_gamma, num_gamma)}


# Performing grid search
grid_search = GridSearchCV(estimator=svm_rbf, param_grid=param_grid, cv=5 , scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_50, y_train_50)

# Best parameters
rbf_best_params_final = grid_search.best_params_
rbf_best_accuracy_final = grid_search.best_score_
rbf_best_params_final

{'C': 0.1, 'gamma': 0.001}

In [55]:
svm_rbf_tunned_f = SVC(kernel='rbf', gamma=rbf_best_params_final['gamma'], C =rbf_best_params_final['C'] )
svm_rbf_tunned_f.fit(X_train_50, y_train_50)

SVC(C=0.1, gamma=0.001)

In [56]:
rbf_t_pred_y_50_f = svm_rbf_tunned_f.predict(X_test_50)

In [57]:
rbf_t_accuracy_f = accuracy_score(y_test_50,rbf_t_pred_y_50_f)
print("Accuracy of SVM with rbf kernel with tunned hyperparameters is ",rbf_t_accuracy_f)

Accuracy of SVM with rbf kernel with tunned hyperparameters is  0.2948717948717949


In [58]:
print(classification_report(y_test_50,rbf_t_pred_y_50_f))

                   precision    recall  f1-score   support

     Ariel_Sharon       0.00      0.00      0.00        14
     Colin_Powell       0.00      0.00      0.00        47
  Donald_Rumsfeld       0.00      0.00      0.00        31
    George_W_Bush       0.29      1.00      0.46        92
Gerhard_Schroeder       0.00      0.00      0.00        21
      Hugo_Chavez       0.00      0.00      0.00        21
   Jacques_Chirac       0.00      0.00      0.00        13
    Jean_Chretien       0.00      0.00      0.00        11
    John_Ashcroft       0.00      0.00      0.00        12
Junichiro_Koizumi       0.00      0.00      0.00        17
  Serena_Williams       0.00      0.00      0.00         8
       Tony_Blair       0.00      0.00      0.00        25

         accuracy                           0.29       312
        macro avg       0.02      0.08      0.04       312
     weighted avg       0.09      0.29      0.13       312



C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bharm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
